# REGARDING GPU USAGE
**RUNNING AN RTX 3090 ON MY PERSONAL MACHINE**
**IT IS RUNNING WINDOWS 10 AND I SPENT A LOT OF TIME SETTING THINGS UP WRONG**

If you want to run your personal GPU with TensorFlow 2, and you're on Windows, definitely follow directions in the link below
in whatever way suits your needs. In my case, I've set up WSL2 on Windows with the latest NVIDIA drivers for my RTX 3090. I
followed the directions for TensorFlow2 which had me install the Ubuntu distro on WSL, then set up Miniconda3 for my venv python work.

**If you find instructions that want you to modify your cudnn or cuda took kit versions and dll files, DO NOT DO IT. Microsoft, via WSL,
has provided the ability to manage TensorFlow and GPU training natively through WSL.**

**ANOTHER NOTE:** The instructions don't speak to it. I was receiving an error that mentioned it may not have been built with NUDA node support... The environment.yml file has been updated. We needed to add `cudnn` and `cudatoolkit` to the conda environment. Without this the GPU is recognized but does not appear to be utilized during training.

https://learn.microsoft.com/en-us/windows/ai/directml/gpu-tensorflow-plugin

## We will explore the fundamental concepts of TensorFlow

This notebook will cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (speeds up regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercises

In [12]:
# Tensorflow logging info that is non-critical is annoying
# using this to reduce clutter
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [13]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__);

2.10.0


In [14]:

# Creating tensors with tf.constant()
scalar = tf.constant(7);
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [15]:
# Check the number of dimensions of a tensor (ndim for num of dimensions)
scalar.ndim

0

In [16]:
# create a vector
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [17]:
# Check the dimensions of our vector
vector.ndim

1

In [18]:
# Create a matrix (array with greater than 1 dimension)
matrix = tf.constant([[10, 7],
                     [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [19]:
# Dimensions in matrix
matrix.ndim

2

In [20]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16)

another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [21]:
# Dimensions of another_matrix
another_matrix.ndim

2

In [22]:
# Create tensor
tensor = tf.constant([[[1, 2, 3,],
                       [4, 5, 6]],
                      [[7, 8, 9],
                       [10,11, 12]],
                      [[13,14,15],
                       [16,17,18]]])

tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [23]:
# dimensions of tensor
tensor.ndim

3

Definitions -
* Scalar - A single number
* Vector - A number with direction (wind speed and direction)
* Matrix - A 2-dimensional array of numbers
* Tensor -An n-dimensional array of numbers (when n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector...) 

### Creating tensors with `tf.variable()`

In [24]:
# Create tensors as above except with tf.variable()
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [25]:
# Try to change an element in the changeable tensor
# changeable_tensor[0] = 7
# changeable_tensor

In [26]:
# Let's try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

2023-01-01 11:20:12.824674: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-01 11:20:12.824742: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 83006 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)


<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [27]:
# Let's try to change the unchangeable tensor
# unchangeable_tensor[0].assign(7)
# unchangeable_tensor

`tf.Variable()` behaves like the `let` keyword in JavaScript. `tf.constant()` behaves like `const` 

**NOTE:** Rarely in practice do you need to decide this. It is typically abstracted away from the framework.

### Creating random tensors

Random tensors are tensors of some arbitrary size which contain random numbers.

In [28]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed so randoms are reproducible
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))

# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in a tensor

In [29]:
# Shuffle a tensor
# tf.random.set_seed(42)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])

# Shuffle our tensor
shuffled = tf.random.shuffle(not_shuffled)

shuffled

# Global vs operation level seeds change the dynamic
# https://www.tensorflow.org/api_docs/python/tf/random/set_seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 2,  5],
       [ 3,  4]], dtype=int32)>

### Other ways to make tensors

In [30]:
# Create a tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [31]:
# Create tensor of all zeros
tf.zeros([5, 4])

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn NumPu arrays into tensors

The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on GPUs and thus speed up the processes.

In [32]:
# You can also turn numpy arrays into TensorFlow tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # create a NumPy array between 1 and 25
numpy_A

# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non-capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [33]:
# convert numpy array into a tensor and manipulate the shape
A = tf.constant(numpy_A, shape=(2, 3, 4))
B = tf.constant(numpy_A)
A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

### Getting information from tensors

When dealing with tensors, these are the attributes to keep in mind:

* Shape
* Rank
* Axis or Dimension
* Size

In [34]:
# Create a rank 4 tensor
rank_4_tensor = tf.zeros([2, 3, 4, 5])
rank_4_tensor


<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [35]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [36]:
# Get various attributes of our tensors
print("Datatype of every element: ", rank_4_tensor.dtype)
print("Number of dimensions (rank): ", rank_4_tensor.ndim)
print("Shape of tensor: ", rank_4_tensor.shape)
print("Elements along the 0 axis: ", rank_4_tensor.shape[0])
print("Elements along the last axis: ", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor).numpy())

Datatype of every element:  <dtype: 'float32'>
Number of dimensions (rank):  4
Shape of tensor:  (2, 3, 4, 5)
Elements along the 0 axis:  2
Elements along the last axis:  5
Total number of elements in our tensor:  120


### Indexing tensors
Tensors can be indexed just like Python list

In [37]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [38]:
# Get the first dimension from each dimension except 
rank_4_tensor[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [39]:
# Create a rank 2 tensor
rank_2_tensor = tf.constant([[10, 7],
                            [3, 4]])

rank_2_tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [40]:
# Get the last item of each row
rank_2_tensor[:, -1]

2023-01-01 11:20:14.041719: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-01 11:20:14.041767: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 83006 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)


<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [41]:
# Add in extra dimensions to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]

rank_3_tensor

2023-01-01 11:20:14.050399: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-01 11:20:14.050445: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 83006 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)


<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [42]:
# alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # -1 means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [43]:
tf.expand_dims(rank_2_tensor, axis=0) # expands 0-axis at the front

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]], dtype=int32)>

### Manipulating tensors (tensor operations)

Basic Operations

`+`, `-`, `*`, `/`


In [44]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10,7], 
                      [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [45]:
# Original tensor unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [46]:
# Multiplication also works
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [47]:
# Subtraction
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [48]:
# We can use the tensflow built-in function
# exists as tf.math.multiply() but has the alias tf.multiply()
# For speedy gpu speed up, remember to do all work using tf libary for calculations
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

**Matrix Multiplication**

In machine learning, matrix multiplcation is one of the most common tensor operations

Two things are true for matrix multiplication:
* Inner dimensions of operands must match
* The resulting matrix has the shape of the outside dimensions of the two operands

In [49]:
# matrix multiplication in tensorflow
# matrix multiplication uses the dot product
tf.matmul(tensor, tensor)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [50]:
# Multiplying two tensors together using `*` does element-wise multiplication
# 10 x 10 = 100, etc
tensor * tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  49],
       [  9,  16]], dtype=int32)>

In [51]:
# matrix multiplication with Python operator @
# notice this is dot product as well
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [52]:
# create a tensor of 3,2 shape
X = tf.constant([[1, 2],
                 [3, 4],
                 [5, 6]])

# create another tensor of same 3,2 shape
Y = tf.constant([[7, 8],
                 [9, 10],
                 [11, 12]])

X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [53]:
# try to matrix multiply tensors of same shape
# X @ Y # incompatible shapes
# tf.matmul(X, Y) # still incompatible

**Resource:** Matrix multiplcation can be found here - https://www.mathsisfun.com/algebra/matrix-multiplying.html

In [54]:
# Let's change the shape of Y
tf.reshape(Y, shape=(2,3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [55]:
Y

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 7,  8],
       [ 9, 10],
       [11, 12]], dtype=int32)>

In [56]:
# Try to use matrix multiplication of X and Y
X @ tf.reshape(Y, shape=(2,3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [57]:
# try tf.matmul
tf.matmul(X, tf.reshape(Y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [58]:
# try it with reshaping X
# We see below that the 2,3 and 3,2 shapes resulted in a 2x2 matrix, differing from the reshaping of Y
tf.matmul(tf.reshape(X, shape=(2,3)), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

In [59]:
# reshape with transpose
X, tf.transpose(X), tf.reshape(X, shape=(2,3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [60]:
# Try matrix multiplication with transpose rather than reshape
tf.matmul(tf.transpose(X), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

**The dot product**

Matrix multiplication is also referred to as the dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`
* `@`

In [61]:
# Perform the dot product on X and Y, this requires transposition of one of them so the shapes are correct
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [62]:
# as above but transposing Y
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [63]:
# Perform matrix multiplication between X and Y
tf.matmul(X, tf.reshape(Y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [64]:
# Check values of Y, reshape Y and transpose Y
print("Normal Y: ")
print(Y, "\n") # \n is a new line character

print("Y reshaped to (2, 3): ") # reshape appears to maintain original order of elements
print(tf.reshape(Y, (2,3), "\n"))

print("Y transposed: ") # transposed causing axes to be flipped
print(tf.transpose(Y))

Normal Y: 
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2, 3): 
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32)
Y transposed: 
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


In [65]:
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

**NOTE:** Generally, when performing matrix multiplication on two tensors and one axis doesn't line up, you will generally transpose one of the tensors rather than reshaping.

### Changing the datatype of a tensor

float32 is the default data type. However, modern NVIDIA hardware, and likely some others, are faster at processing data using float16 data type. The numeric value relates to the number of bits stored in memory, lower number = lower memory required.

**Resource:** https://www.tensorflow.org/guide/mixed_precision#:~:text=The%20float16%20data%20type%20has,underflow%20are%20not%20a%20problem.

In [66]:
# Create a new tensor with a default datatype, tf.float32
B = tf.constant([1.7, 7.4])
B.dtype

tf.float32

In [67]:
# integers are of type tf.int32 type
C = tf.constant([7,10])
C.dtype

tf.int32

In [68]:
# change from float32 to float16 (reduced precision)
D = tf.cast(B, dtype=tf.float16)
D.dtype

tf.float16

In [69]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.int32)
E.dtype

tf.int32

In [70]:
# convert int16 to float16
E_float16 = tf.cast(E, dtype=tf.int16)
E_float16

<tf.Tensor: shape=(2,), dtype=int16, numpy=array([ 7, 10], dtype=int16)>

### Aggregating tensors

Aggregating tensors = condensing them from multiple values into a smaller amount of values

In [71]:
# Create tensor with negative values
D = tf.constant([-7, -10])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10], dtype=int32)>

In [72]:
# create absolute value tensor
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

The various forms of aggregation we cover:
* Get the minimum
* Get the maximum
* Get the mean
* Get the sum

In [73]:
# Create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size=50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([53, 46, 14, 60, 52, 75,  9, 97, 33, 36, 14, 22, 13, 95, 60, 98, 86,
        9, 15, 79, 59, 33, 34, 16, 91, 96, 42, 92, 75, 50, 13, 19, 50, 25,
       60, 25, 86, 53,  1, 53, 37, 38, 54, 86, 55, 14, 52,  6, 74, 41])>

In [74]:
# Check size, shape, and dimensions of our randomly created tensor
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [75]:
# find minimum
tf.reduce_min(E).numpy()

1

In [76]:
# find maximum
tf.reduce_max(E).numpy()

98

In [77]:
# find mean
tf.reduce_mean(E).numpy()

47

In [78]:
# find sum
tf.reduce_sum(E).numpy()

2396

In [79]:
# Find the variance of our tensor
# import tensorflow_probability as tfp <--- doesn't work with this env's version of tensorflow, ignoring as it's largely a non-issue
# tfp.stats.variance(E)

# Turns out I used an unnecessary method
tf.math.reduce_variance(tf.cast(E, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=801.9936>

In [80]:
# Find standard deviation
tf.math.reduce_std(tf.cast(E, dtype=tf.float32))



<tf.Tensor: shape=(), dtype=float32, numpy=28.31949>

## Find the positional maximum and minimum of a tensor

In [81]:
# Create new tensor for finding minima and maxima
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [82]:
# Find the positional maximum
# returns the index of the largest value
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [83]:
# index on our largest value
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [84]:
# Find the max value of F
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [85]:
# Check for equality
F[tf.argmax(F)] == tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [86]:
# find positional minimum
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [87]:
# find minimum value using positional minimum index
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

### Squeezing a tensor (removing all single dimensions)

In [88]:
# create a tensor to get started
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1,1,1,1,50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [89]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [90]:
G_squeezed = tf.squeeze(G)
G_squeezed

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

### One-hot encoding tensors

In [91]:
# Create a list of indices
some_list = [0,1,2,3] # could be red, green, blue, purple

# one-hot encode out indices
tf.one_hot(some_list, depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [92]:
# specify customer values for one hot encoding
# This is silly
tf.one_hot(some_list, depth=4, on_value="Yo, I love deep learning", off_value="I also like to dance")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'Yo, I love deep learning', b'I also like to dance',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'Yo, I love deep learning',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'I also like to dance',
        b'Yo, I love deep learning', b'I also like to dance'],
       [b'I also like to dance', b'I also like to dance',
        b'I also like to dance', b'Yo, I love deep learning']],
      dtype=object)>

### Squaring, log, sqrt

In [93]:
# Create a new tensor
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [94]:
# square it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [95]:
# Square root it
# method requires non-int type (float works)
tf.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.2360678, 2.4494896,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [96]:
# log
# method also required non-int (float works)
tf.math.log(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.6094378, 1.7917594,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Tensors and NumPy

TensorFlow interacts beautifully with NumPy arrays

In [97]:
# Create a tensor directly from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [98]:
# convert our tensor back to a NumPy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [99]:
# Convert tensor J to a NumPy array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [100]:
J = tf.constant([3.])
J.numpy()[0]

3.0

In [101]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])

# check datatypes
# numpy array conversion defaults to tf.float64 data type as opposed to the native tensor datatype of float32
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### Finding access to GPUs

In [102]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [103]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2023-01-01 11:20:15.998289: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-01 11:20:15.998334: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 83006 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10037980005040372127
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 87038329344
 locality {
   bus_id: 1
 }
 incarnation: 15082646128839832430
 physical_device_desc: "device: 0, name: DML, pci bus id: <undefined>"
 xla_global_id: -1]

In [104]:
!nvidia-smi

Sun Jan  1 11:20:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.65       Driver Version: 527.56       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:04:00.0  On |                  N/A |
|  0%   45C    P8    38W / 350W |    754MiB / 24576MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [105]:
device = tf.device('/GPU:0')
device

In [106]:
physical_devices = tf.config.list_physical_devices()
physical_devices

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]